In [126]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import json
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

In [127]:
#define function to print a few job information
def print_info(job):
    
    print('\n')    
    errors = ['pilot','brokerage','taskbuffer','sup','jobdispatcher','ddm','exe']
    #print(job)
    
    print('pandaid ' + str(job['_source']['pandaid']))
    print('modificationtime ' + str(job['_source']['modificationtime']))
    print('site '    + str(job['_source']['computingsite']))
    
    for e in errors:
        errorcode = e + 'errorcode'
        exitcode = job['_source'][errorcode] 
      
        if exitcode:
            print(errorcode + ' ' + str(exitcode))  
            errordiag = e +'errordiag'   
            print(j['_source'][errordiag])   


In [128]:
#define function to debug job information
def debug(job):
        
    errors = ['pilot','brokerage','taskbuffer','sup','jobdispatcher','ddm','exe']
    #print(job)
    pandaid=int(job['_source']['pandaid'])
    
    if (pandaid==3155163997): #3186999624):  #3155206788):
        print('pandaid ' + str(job['_source']['pandaid']))
        print('modificationtime ' + str(job['_source']['modificationtime']))
        print('site '    + str(job['_source']['computingsite']))
    
        for e in errors:
            errorcode = e + 'errorcode'
            exitcode = job['_source'][errorcode] 
      
            if exitcode:
                print(errorcode + ' ' + str(exitcode))  
                errordiag = e +'errordiag'   
                print(j['_source'][errordiag])   


In [129]:
#define function to filter on time
def time_filter(indices, last_days=1, pattern=''):
    if last_days == 0:
        return ["jobs_archive_*"]
    filtered = []
    if pattern:
        for i in indices:
            if pattern in i:
                filtered.append(i.rstrip())
        return filtered
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(r'jobs_archive_', '', i).rstrip()
        #print(day)
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if diff.days < last_days:
            filtered.append(i.rstrip())
    return filtered

In [130]:
#get job archive indices from ES
indices = es.cat.indices(index="jobs_archive_*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
if 'jobs_archive_2016_status' in indices:
    indices.remove('jobs_archive_2016_status')
#remove data due to central problem
if 'jobs_archive_2016-12-29' in indices:
    indices.remove('jobs_archive_2016-12-29')
#print(indices)

In [131]:
# use e.g. last_days=7 or pattern='2016-02' (no wildcard !)
ind = time_filter(indices, last_days=60, pattern='')
ind = ','.join(ind)
print(ind)


jobs_archive_2016-12-27,jobs_archive_2016-12-28,jobs_archive_2016-12-30,jobs_archive_2016-12-31,jobs_archive_2017-01-01,jobs_archive_2017-01-02,jobs_archive_2017-01-03,jobs_archive_2017-01-04,jobs_archive_2017-01-05,jobs_archive_2017-01-06,jobs_archive_2017-01-07,jobs_archive_2017-01-08,jobs_archive_2017-01-09,jobs_archive_2017-01-10,jobs_archive_2017-01-11,jobs_archive_2017-01-12,jobs_archive_2017-01-13,jobs_archive_2017-01-14,jobs_archive_2017-01-15,jobs_archive_2017-01-16,jobs_archive_2017-01-17,jobs_archive_2017-01-18,jobs_archive_2017-01-19,jobs_archive_2017-01-20,jobs_archive_2017-01-21,jobs_archive_2017-01-22,jobs_archive_2017-01-23,jobs_archive_2017-01-24,jobs_archive_2017-01-25,jobs_archive_2017-01-26,jobs_archive_2017-01-27,jobs_archive_2017-01-28,jobs_archive_2017-01-29,jobs_archive_2017-01-30,jobs_archive_2017-01-31,jobs_archive_2017-02-01,jobs_archive_2017-02-02,jobs_archive_2017-02-03,jobs_archive_2017-02-04,jobs_archive_2017-02-05,jobs_archive_2017-02-06,jobs_archive_201

In [132]:
#Try either simple query
query = "jobstatus:failed AND processingtype:gangarobot-pft"

#or add it to body
body = {
    "size": 2,
    "_source": ["computingsite", "pandaid", "modificationtime", "*errorcode", "*errordiag"], #returns only certain fields
    "query": { 
        "bool" : {
            "filter" : [
                { "match" : { "processingtype":"gangarobot-pft" } },
                { "match" : { "jobstatus":"failed" } },
              ],
           #"must" :                 
           #     {"match": {"processingtype":"gangarobot-pft"}},
           # "should" :
           #     {"match": {"jobstatus":"failed"}}
     
        }
    }}

In [133]:
#test query with search -> gets only a limited number of records (size=x). Cannot be used for big searches x>10000
res = es.search(index=ind, q=query, size = 2, request_timeout=600, 
                _source=["computingsite", "pandaid", "modificationtime", "*errorcode", "*errordiag"])
#res = es.search(index=ind, body=body, request_timeout=600)

jobs = res['hits']['hits']

#print(json.dumps(jobs))
#print a few information
for j in jobs:
    print_info(j)
    
#save to file
with open('data_few.txt', 'w') as outfile:  
    json.dump(jobs, outfile)
outfile.close()    



pandaid 3162626191
modificationtime 2017-01-03T00:01:52
site UKI-SCOTGRID-DURHAM-ARC
piloterrorcode 1008
LRMS error: (91) Job failed
exeerrorcode 91
LRMS error: (91) Job failed


pandaid 3162652373
modificationtime 2017-01-03T00:01:36
site INFN-T1_TEST
piloterrorcode 1099
STAGEIN FAILED: Get error: Staging input file failed: lfn=EVNT.04972714._000035.pool.root.1, error=Copy operation failed [is_stagein=True]: LOCAL-SETUP> NO PANDA RESOURCE FOUND
AUTO-SETUP> INFO: CUSTOM_SITE_NAME=, ATLAS_SITE_NAME=, SITE_NAME=INFN-T1,


In [134]:
#count hits
count =(es.count(index=ind, q=query, request_timeout=600))
print('hits: '+str(count['count']))

#use scroll to get all hits (allows unlimted result queries)
scroll = scan(es, index=ind, q=query, scroll='5m', timeout="5m", size=100, _source=["computingsite", "pandaid", "modificationtime", "*errorcode", "*errordiag"])

outfile = open('data.txt', 'w') 

#write results to file
i = 0   
jobs = []
for result in scroll:
    #print(result['_source']['pandaid'])
    #print_info(result)
    jobs.append(result)
    #if i<1: print_info(result)
    i = i+1
    if not i%10000:  print('processing hit '+str(i)+'...')

json.dump(jobs, outfile)         
outfile.close()
print('saved '+str(i)+' results') 

#check written results
infile = open('data.txt', 'r')
savedjobs = json.load(infile)
print('check: reading in '+str(len(savedjobs))+' hits')
infile.close()

print_info(savedjobs[0])
print_info(jobs[0])

hits: 234549
processing hit 10000...
processing hit 20000...
processing hit 30000...
processing hit 40000...
processing hit 50000...
processing hit 60000...
processing hit 70000...
processing hit 80000...
processing hit 90000...
processing hit 100000...
processing hit 110000...
processing hit 120000...
processing hit 130000...
processing hit 140000...
processing hit 150000...
processing hit 160000...
processing hit 170000...
processing hit 180000...
processing hit 190000...
processing hit 200000...
processing hit 210000...
processing hit 220000...
processing hit 230000...
saved 234549 results
check: reading in 234549 hits


pandaid 3186999624
modificationtime 2017-01-15T00:41:25
site RRC-KI-T1_TEST
piloterrorcode 1099
STAGEIN FAILED: Get error: Staging input file failed: lfn=EVNT.04972714._000035.pool.root.1, error=Copy operation failed [is_stagein=True]: LOCAL-SETUP> NO PANDA RESOURCE FOUND
AUTO-SETUP> INFO: CUSTOM_SITE_NAME=, ATLAS_SITE_NAME=, SITE_NAME=INFN-T1,


pandaid 3186999624


In [125]:
i = 0
for j in jobs:
    debug(jobs[i])
    i = i+1
    if not i%10000:  print('processing hit '+str(i)+'...')

processing hit 10000...
processing hit 20000...
processing hit 30000...
processing hit 40000...
processing hit 50000...
processing hit 60000...
processing hit 70000...
processing hit 80000...
processing hit 90000...
processing hit 100000...
processing hit 110000...
processing hit 120000...
processing hit 130000...
processing hit 140000...
processing hit 150000...
processing hit 160000...
processing hit 170000...
processing hit 180000...
processing hit 190000...
processing hit 200000...
processing hit 210000...
processing hit 220000...
processing hit 230000...


In [31]:
#Example of more refined queries
#better to use filters if the score is not needed (faster)
s = {
    "size": 1,
    "_source": ["computingsite", "pandaid", "modificationtime", "*errorcode", "*errordiag"], #returns only certain fields
    "query": { 
        "bool" : {
            "filter" : [
                { "match" : { "processingtype":"gangarobot-pft" } },
                { "match" : { "jobstatus":"failed" } },
              ],
           #"must" :                 
           #     {"match": {"processingtype":"gangarobot-pft"}},
           # "should" :
           #     {"match": {"jobstatus":"failed"}}
     
        }
    },
    "aggs": {  #build aggregation
        "models": 
            {"terms": { "field": "jobstatus" } }
    }    
}

res = es.search(index=ind, body=s, request_timeout=12000)

print(res) 

jobs = res['hits']['hits']
print('check: reading in '+str(len(jobs))+' hits')
#print_info(jobs[0])
print(jobs[0])

{'aggregations': {'models': {'sum_other_doc_count': 0, 'doc_count_error_upper_bound': 0, 'buckets': [{'key': 'failed', 'doc_count': 141998}]}}, 'took': 14, 'hits': {'total': 141998, 'max_score': 0.0, 'hits': [{'_id': '3124983243', '_type': 'jobs_data', '_source': {'taskbuffererrorcode': 0, 'jobdispatchererrorcode': 0, 'ddmerrordiag': None, 'computingsite': 'ATLAS_OPP_OSG-MIT_CMS', 'brokerageerrorcode': 0, 'exeerrorcode': 0, 'modificationtime': '2016-12-14T02:46:42', 'brokerageerrordiag': None, 'superrorcode': 0, 'pandaid': 3124983243, 'superrordiag': None, 'jobdispatchererrordiag': None, 'piloterrordiag': 'Job killed by signal 15: Signal handler has set job result to FAILED, ec = 1201', 'ddmerrorcode': 0, 'piloterrorcode': 1201, 'exeerrordiag': None, 'taskbuffererrordiag': None}, '_score': 0.0, '_index': 'jobs_archive_2016-12-14'}]}, '_shards': {'total': 155, 'failed': 0, 'successful': 155}, 'timed_out': False}
check: reading in 1 hits
{'_id': '3124983243', '_type': 'jobs_data', '_sour